# Building the 12-factor agent template from scratch in Python

Steps to start from a bare Python repo and build up a 12-factor agent. This walkthrough will guide you through creating a Python agent that follows the 12-factor methodology with BAML.

## Chapter 0 - Hello World

Let's start with a basic Python setup and a hello world program.

This guide will walk you through building agents in Python with BAML.

We'll start simple with a hello world program and gradually build up to a full agent.

For this notebook, you'll need to have your OpenAI API key saved in Google Colab secrets.


Here's our simple hello world program:

In [ ]:
# ./walkthrough/00-main.py
def hello():
    print('hello, world!')

def main():
    hello()

Let's run it to verify it works:

In [ ]:
main()

## Chapter 1 - CLI and Agent Loop

Now let's add BAML and create our first agent with a CLI interface.

In this chapter, we'll integrate BAML to create an AI agent that can respond to user input.

## What is BAML?

BAML (Boundary Markup Language) is a domain-specific language designed to help developers build reliable AI workflows and agents. Created by [BoundaryML](https://www.boundaryml.com/) (a Y Combinator W23 company), BAML adds the engineering to prompt engineering.

### Why BAML?

- **Type-safe outputs**: Get fully type-safe outputs from LLMs, even when streaming
- **Language agnostic**: Works with Python, TypeScript, Ruby, Go, and more
- **LLM agnostic**: Works with any LLM provider (OpenAI, Anthropic, etc.)
- **Better performance**: State-of-the-art structured outputs that outperform even OpenAI's native function calling
- **Developer-friendly**: Native VSCode extension with syntax highlighting, autocomplete, and interactive playground

### Learn More

- 📚 [Official Documentation](https://docs.boundaryml.com/home)
- 💻 [GitHub Repository](https://github.com/BoundaryML/baml)
- 🎯 [What is BAML?](https://docs.boundaryml.com/guide/introduction/what-is-baml)
- 📖 [BAML Examples](https://github.com/BoundaryML/baml-examples)
- 🏢 [Company Website](https://www.boundaryml.com/)
- 📰 [Blog: AI Agents Need a New Syntax](https://www.boundaryml.com/blog/ai-agents-need-new-syntax)

BAML turns prompt engineering into schema engineering, where you focus on defining the structure of your data rather than wrestling with prompts. This approach leads to more reliable and maintainable AI applications.

### Note on Developer Experience

BAML works much better in VS Code with their official extension, which provides syntax highlighting, autocomplete, inline testing, and an interactive playground. However, for this notebook tutorial, we'll work with BAML files directly without the enhanced IDE features.

First, let's set up BAML support in our notebook.


### BAML Setup

Don't worry too much about this setup code - it will make sense later! For now, just know that:
- BAML is a tool for working with language models
- We need some special setup code to make it work nicely in Google Colab
- The `get_baml_client()` function will be used to interact with AI models

In [ ]:
!pip install baml-py==0.202.0 pydantic

In [ ]:
import subprocess
import os

# Try to import Google Colab userdata, but don't fail if not in Colab
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

def baml_generate():
    try:
        result = subprocess.run(
            ["baml-cli", "generate"],
            check=True,
            capture_output=True,
            text=True
        )
        if result.stdout:
            print("[baml-cli generate]\n", result.stdout)
        if result.stderr:
            print("[baml-cli generate]\n", result.stderr)
    except subprocess.CalledProcessError as e:
        msg = (
            f"`baml-cli generate` failed with exit code {e.returncode}\n"
            f"--- STDOUT ---\n{e.stdout}\n"
            f"--- STDERR ---\n{e.stderr}"
        )
        raise RuntimeError(msg) from None

def get_baml_client():
    """
    a bunch of fun jank to work around the google colab import cache
    """
    # Set API key from Colab secrets or environment
    if IN_COLAB:
        os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    elif 'OPENAI_API_KEY' not in os.environ:
        print("Warning: OPENAI_API_KEY not set. Please set it in your environment.")
    
    baml_generate()
    
    # Force delete all baml_client modules from sys.modules
    import sys
    modules_to_delete = [key for key in sys.modules.keys() if key.startswith('baml_client')]
    for module in modules_to_delete:
        del sys.modules[module]
    
    # Now import fresh
    import baml_client
    return baml_client.sync_client.b


In [ ]:
!baml-cli init

Now let's create our agent that will use BAML to process user input.

First, we'll define the core agent logic:


In [ ]:
# ./walkthrough/01-agent.py
import json
from typing import Dict, Any, List

# tool call or a respond to human tool
AgentResponse = Any  # This will be the return type from b.DetermineNextStep

class Event:
    def __init__(self, type: str, data: Any):
        self.type = type
        self.data = data

class Thread:
    def __init__(self, events: List[Dict[str, Any]]):
        self.events = events
    
    def serialize_for_llm(self):
        # can change this to whatever custom serialization you want to do, XML, etc
        # e.g. https://github.com/got-agents/agents/blob/59ebbfa236fc376618f16ee08eb0f3bf7b698892/linear-assistant-ts/src/agent.ts#L66-L105
        return json.dumps(self.events)

# right now this just runs one turn with the LLM, but
# we'll update this function to handle all the agent logic
def agent_loop(thread: Thread) -> AgentResponse:
    b = get_baml_client()  # This will be defined by the BAML setup
    next_step = b.DetermineNextStep(thread.serialize_for_llm())
    return next_step

Next, we need to define the BAML function that our agent will use.

### Understanding BAML Syntax

BAML files define:
- **Classes**: Structured output schemas (like `DoneForNow` below)
- **Functions**: AI-powered functions that take inputs and return structured outputs
- **Tests**: Example inputs/outputs to validate your prompts

This BAML file defines what our agent can do:


In [ ]:
!curl -fsSL -o baml_src/agent.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/./walkthrough/01-agent.baml && cat baml_src/agent.baml

Now let's create our main function that accepts a message parameter:


In [ ]:
# ./walkthrough/01-main.py
def main(message="hello from the notebook!"):
    # Create a new thread with the user's message as the initial event
    thread = Thread([{"type": "user_input", "data": message}])
    
    # Run the agent loop with the thread
    result = agent_loop(thread)
    print(result)

Let's test our agent! Try calling main() with different messages:
- `main("What's the weather like?")`
- `main("Tell me a joke")`
- `main("How are you doing today?")`


In [ ]:
baml_generate()

In [ ]:
main("Hello from the Python notebook!")

## Chapter 2 - Add Calculator Tools

Let's add some calculator tools to our agent.

Let's start by adding a tool definition for the calculator.

These are simple structured outputs that we'll ask the model to
return as a "next step" in the agentic loop.


In [ ]:
!curl -fsSL -o baml_src/tool_calculator.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/./walkthrough/02-tool_calculator.baml && cat baml_src/tool_calculator.baml

Now, let's update the agent's DetermineNextStep method to
expose the calculator tools as potential next steps.


In [ ]:
!curl -fsSL -o baml_src/agent.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/./walkthrough/02-agent.baml && cat baml_src/agent.baml

Now let's update our main function to show the tool call:


In [ ]:
# ./walkthrough/02-main.py
def main(message="hello from the notebook!"):
    # Create a new thread with the user's message
    thread = Thread([{"type": "user_input", "data": message}])
    
    # Get BAML client
    b = get_baml_client()
    
    # Get the next step from the agent - just show the tool call
    next_step = b.DetermineNextStep(thread.serialize_for_llm())
    
    # Print the raw response to show the tool call
    print(next_step)

Let's try out the calculator! The agent should recognize that you want to perform a calculation
and return the appropriate tool call instead of just a message.


In [ ]:
baml_generate()

In [ ]:
main("can you add 3 and 4")

## Chapter 3 - Process Tool Calls in a Loop

Now let's add a real agentic loop that can run the tools and get a final answer from the LLM.

In this chapter, we'll enhance our agent to process tool calls in a loop. This means:
- The agent can call multiple tools in sequence
- Each tool result is fed back to the agent
- The agent continues until it has a final answer

Let's update our agent to handle tool calls properly:


In [ ]:
# ./walkthrough/03-agent.py
import json
from typing import Dict, Any, List

class Thread:
    def __init__(self, events: List[Dict[str, Any]]):
        self.events = events
    
    def serialize_for_llm(self):
        # can change this to whatever custom serialization you want to do, XML, etc
        # e.g. https://github.com/got-agents/agents/blob/59ebbfa236fc376618f16ee08eb0f3bf7b698892/linear-assistant-ts/src/agent.ts#L66-L105
        return json.dumps(self.events)


def agent_loop(thread: Thread) -> str:
    b = get_baml_client()
    
    while True:
        next_step = b.DetermineNextStep(thread.serialize_for_llm())
        print("nextStep", next_step)
        
        if next_step.intent == "done_for_now":
            # response to human, return the next step object
            return next_step.message
        elif next_step.intent == "add":
            thread.events.append({
                "type": "tool_call",
                "data": next_step.__dict__
            })
            result = next_step.a + next_step.b
            print("tool_response", result)
            thread.events.append({
                "type": "tool_response",
                "data": result
            })
            continue
        else:
            raise ValueError(f"Unknown intent: {next_step.intent}")

Now let's update our main function to use the new agent loop:


In [ ]:
# ./walkthrough/03-main.py
def main(message="hello from the notebook!"):
    # Create a new thread with the user's message
    thread = Thread([{"type": "user_input", "data": message}])
    
    # Run the agent loop with full tool handling
    result = agent_loop(thread)
    
    # Print the final response
    print(f"\nFinal response: {result}")

Let's try it out! The agent should now call the tool and return the calculated result:


In [ ]:
baml_generate()

In [ ]:
main("can you add 3 and 4")

You should see the agent:
1. Recognize it needs to use the add tool
2. Call the tool with the correct parameters
3. Get the result (7)
4. Generate a final response incorporating the result

For more complex calculations, we need to handle all calculator operations. Let's add support for subtract, multiply, and divide:


In [ ]:
# ./walkthrough/03b-agent.py
import json
from typing import Dict, Any, List, Union

class Thread:
    def __init__(self, events: List[Dict[str, Any]]):
        self.events = events
    
    def serialize_for_llm(self):
        # can change this to whatever custom serialization you want to do, XML, etc
        # e.g. https://github.com/got-agents/agents/blob/59ebbfa236fc376618f16ee08eb0f3bf7b698892/linear-assistant-ts/src/agent.ts#L66-L105
        return json.dumps(self.events)

def handle_next_step(next_step, thread: Thread) -> Thread:
    result: float
    
    if next_step.intent == "add":
        result = next_step.a + next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread
    elif next_step.intent == "subtract":
        result = next_step.a - next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread
    elif next_step.intent == "multiply":
        result = next_step.a * next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread
    elif next_step.intent == "divide":
        result = next_step.a / next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread

def agent_loop(thread: Thread) -> str:
    b = get_baml_client()
    
    while True:
        next_step = b.DetermineNextStep(thread.serialize_for_llm())
        print("nextStep", next_step)
        
        thread.events.append({
            "type": "tool_call",
            "data": next_step.__dict__
        })
        
        if next_step.intent == "done_for_now":
            # response to human, return the next step object
            return next_step.message
        elif next_step.intent in ["add", "subtract", "multiply", "divide"]:
            thread = handle_next_step(next_step, thread)

Now let's test subtraction:


In [ ]:
main("can you subtract 3 from 4")

Test multiplication:


In [ ]:
main("can you multiply 3 and 4")

Finally, let's test a complex multi-step calculation:


In [ ]:
main("can you multiply 3 and 4, then divide the result by 2 and then add 12 to that result")

Congratulations! You've taken your first step into hand-rolling an agent loop.

Key concepts you've learned:
- **Thread Management**: Tracking conversation history and tool calls
- **Tool Execution**: Processing different tool types and returning results
- **Agent Loop**: Continuing until the agent has a final answer

From here, we'll start incorporating more intermediate and advanced concepts for 12-factor agents.


## Chapter 4 - Add Tests to agent.baml

Let's add some tests to our BAML agent.

In this chapter, we'll learn about BAML testing - a powerful feature that helps ensure your agents behave correctly.

## Why Test BAML Functions?

- **Catch regressions**: Ensure changes don't break existing behavior
- **Document behavior**: Tests serve as living documentation
- **Validate edge cases**: Test complex scenarios and conversation flows
- **CI/CD integration**: Run tests automatically in your pipeline

Let's start with a simple test that checks the agent's ability to handle basic interactions:


In [ ]:
!curl -fsSL -o baml_src/agent.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/./walkthrough/04-agent.baml && cat baml_src/agent.baml

Run the tests to see them in action:


In [ ]:
!baml-cli test

Now let's improve the tests with assertions! Assertions let you verify specific properties of the agent's output.

## BAML Assertion Syntax

Assertions use the `@@assert` directive:
```
@@assert(name, {{condition}})
```

- `name`: A descriptive name for the assertion
- `condition`: A boolean expression using `this` to access the output


In [ ]:
!curl -fsSL -o baml_src/agent.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/./walkthrough/04b-agent.baml && cat baml_src/agent.baml

Run the tests again to see assertions in action:


In [ ]:
!baml-cli test

Finally, let's add more complex test cases that test multi-step conversations.

These tests simulate an entire conversation flow, including:
- User input
- Tool calls made by the agent
- Tool responses
- Final agent response


In [ ]:
!curl -fsSL -o baml_src/agent.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/./walkthrough/04c-agent.baml && cat baml_src/agent.baml

Run the comprehensive test suite:


In [ ]:
!baml-cli test

## Key Testing Concepts

1. **Test Structure**: Each test specifies functions, arguments, and assertions
2. **Progressive Testing**: Start simple, then test complex scenarios
3. **Conversation History**: Test how the agent handles multi-turn conversations
4. **Tool Integration**: Verify the agent correctly uses tools in sequence

With these tests in place, you can confidently modify your agent knowing that core functionality is protected by automated tests!


## Chapter 5 - Multiple Human Tools

In this section, we'll add support for multiple tools that serve to contact humans.


So far, our agent only returns a final answer with "done_for_now". But what if the agent needs clarification?

Let's add a new tool that allows the agent to request more information from the user.

## Why Human-in-the-Loop?

- **Handle ambiguous inputs**: When user input is unclear or contains typos
- **Request missing information**: When the agent needs more context
- **Confirm sensitive operations**: Before performing important actions
- **Interactive workflows**: Build conversational agents that engage users

First, let's update our BAML file to include a ClarificationRequest tool:


In [ ]:
!curl -fsSL -o baml_src/agent.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/./walkthrough/05-agent.baml && cat baml_src/agent.baml

Now let's update our agent to handle clarification requests:


In [ ]:
# ./walkthrough/05-agent.py
import json
from typing import Dict, Any, List, Union

class Thread:
    def __init__(self, events: List[Dict[str, Any]]):
        self.events = events
    
    def serialize_for_llm(self):
        return json.dumps(self.events)

def handle_next_step(next_step, thread: Thread) -> Thread:
    result: float
    
    if next_step.intent == "add":
        result = next_step.a + next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread
    elif next_step.intent == "subtract":
        result = next_step.a - next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread
    elif next_step.intent == "multiply":
        result = next_step.a * next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread
    elif next_step.intent == "divide":
        result = next_step.a / next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread

def agent_loop(thread: Thread, handle_clarification=None) -> str:
    b = get_baml_client()
    
    while True:
        next_step = b.DetermineNextStep(thread.serialize_for_llm())
        print("nextStep", next_step)
        
        thread.events.append({
            "type": "tool_call",
            "data": next_step.__dict__
        })
        
        if next_step.intent == "done_for_now":
            # response to human, return the next step object
            return next_step.message
        elif next_step.intent == "request_more_information":
            if handle_clarification:
                # Get clarification from human
                clarification = handle_clarification(next_step.message)
                thread.events.append({
                    "type": "human_response",
                    "data": clarification
                })
                continue
            else:
                # No handler provided, return the question
                return f"Clarification needed: {next_step.message}"
        elif next_step.intent in ["add", "subtract", "multiply", "divide"]:
            thread = handle_next_step(next_step, thread)
        else:
            raise ValueError(f"Unknown intent: {next_step.intent}")

Finally, let's create a main function that handles human interaction:


In [ ]:
# ./walkthrough/05-main.py
def get_human_input(prompt):
    """Get input from human, handling both Colab and local environments."""
    print(f"\n🤔 {prompt}")
    
    if IN_COLAB:
        # In Colab, use actual input
        response = input("Your response: ")
    else:
        # In local testing, return a fixed response
        response = "I meant to multiply 3 and 4"
        print(f"📝 [Auto-response for testing]: {response}")
    
    return response

def main(message="hello from the notebook!"):
    # Function to handle clarification requests
    def handle_clarification(question):
        return get_human_input(f"The agent needs clarification: {question}")
    
    # Create a new thread with the user's message
    thread = Thread([{"type": "user_input", "data": message}])
    
    # Run the agent loop with clarification handler
    result = agent_loop(thread, handle_clarification)
    
    # Print the final response
    print(f"\n✅ Final response: {result}")

Let's test with an ambiguous input that should trigger a clarification request:


In [ ]:
baml_generate()

In [ ]:
main("can you multiply 3 and FD*(#F&&")

You should see:
1. The agent recognizes the input is unclear
2. It asks for clarification
3. In Colab, you'll be prompted to type a response
4. In local testing, an auto-response is provided
5. The agent continues with the clarified input

## Interactive Testing in Colab

When running in Google Colab, the `input()` function will create an interactive text box where you can type your response. Try different clarifications to see how the agent adapts!

## Key Concepts

- **Human Tools**: Special tool types that return control to the human
- **Conversation Flow**: The agent can pause execution to get human input
- **Context Preservation**: The full conversation history is maintained
- **Flexible Handling**: Different behaviors for different environments


## Chapter 6 - Customize Your Prompt with Reasoning

In this section, we'll explore how to customize the prompt of the agent with reasoning steps.

This is core to [factor 2 - own your prompts](https://github.com/humanlayer/12-factor-agents/blob/main/content/factor-2-own-your-prompts.md)


## Why Add Reasoning to Prompts?

Adding explicit reasoning steps to your prompts can significantly improve agent performance:

- **Better decisions**: The model thinks through problems step-by-step
- **Transparency**: You can see the model's thought process
- **Fewer errors**: Structured thinking reduces mistakes
- **Debugging**: Easier to identify where reasoning went wrong

Let's update our agent prompt to include a reasoning step:


In [ ]:
!curl -fsSL -o baml_src/agent.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/./walkthrough/06-agent.baml && cat baml_src/agent.baml

Now let's test it with a simple calculation to see the reasoning in action:


In [ ]:
baml_generate()

In [ ]:
main("can you multiply 3 and 4")

You should notice in the BAML logs (if enabled) that the model now includes reasoning steps before deciding what to do.

## Advanced Prompt Engineering

You can enhance your prompts further by:
- Adding specific reasoning templates for different tasks
- Including examples of good reasoning
- Structuring the reasoning with numbered steps
- Adding checks for common mistakes

The key is to guide the model's thinking process while still allowing flexibility.


## Chapter 7 - Customize Your Context Window

In this section, we'll explore how to customize the context window of the agent.

This is core to [factor 3 - own your context window](https://github.com/humanlayer/12-factor-agents/blob/main/content/factor-3-own-your-context-window.md)


## Context Window Serialization

How you format your conversation history can significantly impact:
- **Token usage**: Some formats are more efficient
- **Model understanding**: Clear structure helps the model
- **Debugging**: Readable formats help development

Let's implement two serialization formats: pretty-printed JSON and XML.


In [ ]:
# ./walkthrough/07-agent.py
import json
from typing import Dict, Any, List, Union

class Thread:
    def __init__(self, events: List[Dict[str, Any]]):
        self.events = events
    
    def serialize_for_llm(self):
        """Serialize thread events to a pretty-printed format for better readability."""
        # Pretty print the JSON for better debugging
        return json.dumps(self.events, indent=2)
    
    def serialize_as_xml(self):
        """Serialize thread events to XML format for better token efficiency."""
        xml_parts = ["<thread>"]
        
        for event in self.events:
            event_type = event.get("type", "unknown")
            data = event.get("data", {})
            
            if event_type == "user_input":
                xml_parts.append(f"  <user_input>{data}</user_input>")
            elif event_type == "tool_call":
                tool_data = data if isinstance(data, dict) else {}
                intent = tool_data.get("intent", "unknown")
                xml_parts.append(f"  <tool_call intent=\"{intent}\">")
                for key, value in tool_data.items():
                    if key != "intent":
                        xml_parts.append(f"    <{key}>{value}</{key}>")
                xml_parts.append("  </tool_call>")
            elif event_type == "tool_response":
                xml_parts.append(f"  <tool_response>{data}</tool_response>")
            elif event_type == "human_response":
                xml_parts.append(f"  <human_response>{data}</human_response>")
            else:
                xml_parts.append(f"  <{event_type}>{data}</{event_type}>")
        
        xml_parts.append("</thread>")
        return "\n".join(xml_parts)

def handle_next_step(next_step, thread: Thread) -> Thread:
    result: float
    
    if next_step.intent == "add":
        result = next_step.a + next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread
    elif next_step.intent == "subtract":
        result = next_step.a - next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread
    elif next_step.intent == "multiply":
        result = next_step.a * next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread
    elif next_step.intent == "divide":
        result = next_step.a / next_step.b
        print("tool_response", result)
        thread.events.append({
            "type": "tool_response",
            "data": result
        })
        return thread

def agent_loop(thread: Thread, handle_clarification=None, use_xml=False) -> str:
    b = get_baml_client()
    
    while True:
        # Choose serialization format
        if use_xml:
            serialized_thread = thread.serialize_as_xml()
            print("\n📄 Thread (XML format):")
            print(serialized_thread)
        else:
            serialized_thread = thread.serialize_for_llm()
            print("\n📄 Thread (JSON format):")
            print(serialized_thread)
        
        next_step = b.DetermineNextStep(serialized_thread)
        print("\n🤖 Next step:", next_step)
        
        thread.events.append({
            "type": "tool_call",
            "data": next_step.__dict__
        })
        
        if next_step.intent == "done_for_now":
            return next_step.message
        elif next_step.intent == "request_more_information":
            if handle_clarification:
                clarification = handle_clarification(next_step.message)
                thread.events.append({
                    "type": "human_response",
                    "data": clarification
                })
                continue
            else:
                return f"Clarification needed: {next_step.message}"
        elif next_step.intent in ["add", "subtract", "multiply", "divide"]:
            thread = handle_next_step(next_step, thread)
        else:
            raise ValueError(f"Unknown intent: {next_step.intent}")

Now let's create a main function that can switch between formats:


In [ ]:
# ./walkthrough/07-main.py
def main(message="hello from the notebook!", use_xml=True):
    # Function to handle clarification requests
    def handle_clarification(question):
        return get_human_input(f"The agent needs clarification: {question}")
    
    # Create a new thread with the user's message
    thread = Thread([{"type": "user_input", "data": message}])
    
    print(f"🚀 Starting agent with message: '{message}'")
    print(f"📋 Using {'XML' if use_xml else 'JSON'} format for thread serialization")
    
    # Run the agent loop with XML serialization
    result = agent_loop(thread, handle_clarification, use_xml=use_xml)
    
    # Print the final response
    print(f"\n✅ Final response: {result}")

Let's test with JSON format first:


In [ ]:
baml_generate()

In [ ]:
main("can you multiply 3 and 4, then divide the result by 2", use_xml=False)

Now let's try the same with XML format:


In [ ]:
main("can you multiply 3 and 4, then divide the result by 2", use_xml=True)

## XML vs JSON Trade-offs

**XML Benefits**:
- More token-efficient for nested data
- Clear hierarchy with opening/closing tags
- Better for long conversations

**JSON Benefits**:
- Familiar to most developers
- Easy to parse and debug
- Native to JavaScript/Python

Choose based on your specific needs and token constraints!